## Chapter 2 : 머신러닝 프로젝트 처음부터 끝까지

### Train Set과 Test Set으로 구분
  - `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 42)
# train set과 test set을 80 : 20 비율로 split 하고, random seed를 42로 설정

print("train set의 data 개수 : ", len(train_set))
print("test set의 data 개수 : ", len(test_set))

### 계층 샘플링(Stratified Shuffle Split)
  - `StratifiedShuffleSplit`

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
# 분할의 수 : 1, test set : 20%, 난수 초기 값 : 42

# housing Data를 계층에 따라 샘플링
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    print(train_index)

In [ ]:
# test set에서의 소득 카테고리 비율(계층 샘플링 사용)
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

### 누락된 값을 대체
  - `SimpleImputer`

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = "median")

# 중간값이 수치형 특성에서만 계산될 수 있기 때문에 텍스트 특성을 삭제
housing_num = housing.drop("ocean_proximity", axis = 1)

imputer.fit(housing_num)

imputer는 각 features들의 중앙값을 계산해서 statistics_ 속성에 저장합니다. 

In [ ]:
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [ ]:
imputer.statistics_.shape

(8,)

각 특성의 중간 값이 수동으로 계산한 것과 같은지 확인해 보세요:

In [ ]:
housing_num.median().values

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

이제 위에서 학습한 imputer 객체를 사용하여 Train Set에서 누락된 값을 학습한 중앙값으로 바꿀 수 있습니다.

In [ ]:
X = imputer.transform(housing_num)
X

array([[-1.2146e+02,  3.8520e+01,  2.9000e+01, ...,  2.2370e+03,
         7.0600e+02,  2.1736e+00],
       [-1.1723e+02,  3.3090e+01,  7.0000e+00, ...,  2.0150e+03,
         7.6800e+02,  6.3373e+00],
       [-1.1904e+02,  3.5370e+01,  4.4000e+01, ...,  6.6700e+02,
         3.0000e+02,  2.8750e+00],
       ...,
       [-1.2272e+02,  3.8440e+01,  4.8000e+01, ...,  4.5800e+02,
         1.7200e+02,  3.1797e+00],
       [-1.2270e+02,  3.8310e+01,  1.4000e+01, ...,  1.2080e+03,
         5.0100e+02,  4.1964e+00],
       [-1.2214e+02,  3.9970e+01,  2.7000e+01, ...,  6.2500e+02,
         1.9700e+02,  3.1319e+00]])

In [ ]:
housing_tr = pd.DataFrame(X, 
                          columns = housing_num.columns,
                          index = housing_num.index)

housing_tr.loc[sample_incomplete_rows.index.values] # total_bedrooms의 NaN 값들이 모두 중앙값인 433.0으로 대체

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-122.08,37.88,26.0,2947.0,433.0,825.0,626.0,2.9330
10915,-117.87,33.73,45.0,2264.0,433.0,1970.0,499.0,3.4193
19150,-122.70,38.35,14.0,2313.0,433.0,954.0,397.0,3.7813
4186,-118.23,34.13,48.0,1308.0,433.0,835.0,294.0,4.2891
16885,-122.40,37.58,26.0,3281.0,433.0,1145.0,480.0,6.3580


### 범주형 변수 변환 1
  - `OrdinalEncoder`
    - 순서척도의 경우에는 유용하게 사용이 가능하나, 명목척도에 대해서는 사용 시 문제점이 발생

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

### 범주형 변수 변환 2
  - `OneHotEncoder`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

`OneHotEncoder`는 기본적으로 희소 행렬을 반환합니다. 필요하면 `toarray()` 메서드를 사용해 밀집 배열로 변환할 수 있습니다:

In [ ]:
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

또는 `OneHotEncoder`를 만들 때 `sparse=False`로 지정할 수 있습니다:

In [ ]:
cat_encoder = OneHotEncoder(sparse = False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

### 사용자 정의 변환기
  - `BaseEstimator`, `TransformerMixin`

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # *args 또는 **kargs 없음
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # 아무것도 하지 않습니다
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, 
                         rooms_per_household, 
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, 
                         rooms_per_household, 
                         population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.to_numpy())

### 변환 파이프라인
  - `Pipeline`

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # 결측치값을 median으로 치환해주는 Imputer
        ('attribs_adder', CombinedAttributesAdder()), # 추가적인 feature를 만들어주는 Attribs_adder
        ('std_scaler', StandardScaler()), # 모든 features들의 스케일을 표준화 시켜주는 StandardScaler
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

### 선형 회귀 모형
  - `LinearRegression`

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

### 회귀모델 성능 예측(MSE / MAE)
  - `mean_squared_error`
  - `mean_absolute_error`

In [ ]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(housing_labels, housing_predictions, squared = False)
lin_mse

from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

### 교차검증을 사용한 평가
  - `cross_val_score`

### 의사결정나무 회귀모델
  - `DecisionTreeRegressor`

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state = 42)

scores = cross_val_score(estimator = tree_reg,                # 의사결정나무 회귀모델 사용
                         X = housing_prepared,                # X값 (train data)
                         y = housing_labels,                  # y값 (target)
                         scoring = "neg_mean_squared_error",  # - MSE (MSE 값에 - 부호를 붙힘)
                         cv = 10)                             # Cross Vaildation

tree_rmse_scores = np.sqrt(-scores)

### 랜덤포레스트 회귀모델
  - `RandomForestRegressor`

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators = 100, random_state = 42)
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(random_state=42)

### 그리드 탐색
  - `GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

# 총 18개의 하이퍼파라미터 조합을 테스트 합니다. 
param_grid = [
    # 12(= 3 x 4)개의 하이퍼파라미터 조합을 시도합니다.
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # bootstrap은 False로 하고 6(= 2 x 3)개의 조합을 시도합니다.
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state = 42)

# 5개의 폴드로 훈련하면 총 (12 + 6) x 5 = 90번의 훈련이 일어납니다.
grid_search = GridSearchCV(forest_reg, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error',
                           return_train_score = True)

grid_search.fit(housing_prepared, housing_labels)

### 랜덤 탐색
  - `RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low = 1, high = 200),
        'max_features': randint(low = 1, high = 8),
    }

forest_reg = RandomForestRegressor(random_state = 42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions = param_distribs,
                                n_iter = 10, cv = 5, scoring = 'neg_mean_squared_error', random_state = 42)
rnd_search.fit(housing_prepared, housing_labels)

## Chapter 3 : 분류 (Classification)

### 이진 분류기 (Binary Classifier)
  - `SGDClassifier`

In [ ]:
from sklearn.linear_model import SGDClassifier 
# SGD = Stochastic Gradient Decent 
# Gradient Decent의 개념은 뒤에 더 자세히 다룹니다

sgd_clf = SGDClassifier(max_iter = 1000, tol = 1e-3, random_state = 42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
some_digit = X[0]

sgd_clf.predict([some_digit])

### 오차행렬 (Confusion Matrix)
  - `confusion_matrix`

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)

### 정밀도, 재현율, F1 Score
  - 정밀도 : `precision_score`
  - 재현율 : `recall_score`
  - F1 Score : `f1_score`

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정밀도
precision_score(y_train_5, y_train_pred)

# 재현율
recall_score(y_train_5, y_train_pred)

# F1 Score
f1_score(y_train_5, y_train_pred)

### ROC Curve
  - `roc_curve`

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

### ROC Curve의 아래 면적 (AUC)
  - `roc_auc_curve`

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

### 서포트 벡터 머신 분류기
  - `SVC`

In [ ]:
# 서포트 벡터 머신의 분류기 사용
from sklearn.svm import SVC

svm_clf = SVC(gamma = "auto", random_state = 42)
svm_clf.fit(X_train[:1000], y_train[:1000]) # y_train_5이 아니라 y_train입니다
svm_clf.predict([some_digit])

### K-최근접이웃 분류기
  - `KNeighborsClassifier`

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)